In [16]:
import numpy as np
# coords = np.array([[0,0], [1,0], [3,5], [10,20]])

In [10]:
# converts coordinate configuration into angles configuration
def coords2angle(coords):   
    thetas = []
    lengths = []
    for ind in range(len(coords)-1):
        edge = coords[ind+1] - coords[ind]
        length = np.linalg.norm(edge)
        lengths.append(length)
        theta = np.arccos(edge[0]/length)
        if edge[1]<0:
            theta+=np.pi
        thetas.append(theta)
    return thetas, lengths

In [43]:
# converts angles configuration back to coordinate configuration
import cmath
def angle2coords(angles, lengths):
    coords = [[0,0]]
    for ind in range(len(angles)):
        # Get the displacement of the new coord as a complex number
        coord = lengths[ind] * np.exp(complex(0, angles[ind]))
        # Convert the complex number to a coord and offset it with the last coord
        coord = [coords[ind][0] + np.real(coord), coords[ind][1] + np.imag(coord)]
        coords.append(coord)
    return coords

In [21]:
import cmath
np.exp(complex(0, np.pi))

(-1+1.2246467991473532e-16j)

In [13]:
# computes `ENERGY` of a configuration
def energy(coords):
    E = 0
    for ind in range(len(coords)-1):
        u = coords[ind]
        v = coords[ind+1]
        edge = v - u
        for curr in range(len(coords)):
            if curr==ind or curr==ind+1:
                continue
            w = coords[curr]
            E+=1/(np.linalg.norm(u-w)+np.linalg.norm(v-w)-np.linalg.norm(u-v))**2
    return E

In [15]:
energy(coords)

587.786229230512

In [2]:
coords2angle(coords)

([0.0, 1.1902899496825317, 1.1341691669813554],
 [1.0, 5.385164807134504, 16.55294535724685])

In [14]:
coords = np.array([[0.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 0.25],
       [0.75, 0.25],
       [0.75, 0.75],
       [0.25, 0.75],
       [0.25, 0.5 ],
       [0.5 , 0.5 ]])
coords

array([[0.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 0.25],
       [0.75, 0.25],
       [0.75, 0.75],
       [0.25, 0.75],
       [0.25, 0.5 ],
       [0.5 , 0.5 ]])

In [45]:
print(coords)
angles, lengths = coords2angle(coords)
angle2coords(angles, lengths)

[[0.   0.  ]
 [1.   0.  ]
 [1.   1.  ]
 [0.   1.  ]
 [0.   0.25]
 [0.75 0.25]
 [0.75 0.75]
 [0.25 0.75]
 [0.25 0.5 ]
 [0.5  0.5 ]]


[[0, 0],
 [1.0, 0.0],
 [1.0, 1.0],
 [0.0, 1.0000000000000002],
 [-1.3777276490407724e-16, 0.2500000000000002],
 [0.7499999999999999, 0.2500000000000002],
 [0.7499999999999999, 0.7500000000000002],
 [0.2499999999999999, 0.7500000000000003],
 [0.24999999999999983, 0.5000000000000003],
 [0.49999999999999983, 0.5000000000000003]]